# Webscraping

In this session, we will do some exercises about webscraping. Webscraping is the evil twin of API calls, seen last week. At the core of webscraping, there is a game: Usually, the website operators wants to prevent you from scraping her site as she would lose the ad revenues. It is therefore usually a fairly complex work where you have to outsmart somebody who makes part of her livelihood out of the data provided on the site.

Moreover, scrapers are tipically much faster than humans and put therefore much more strain on the servers than the normal human user and require therefore more computing power on the part of the data provider. This long and boring introduction to say: please scrape responsibly.

## Nobel purpose

We know that the Nobel Prize in economics does not exist. The closest thing we have is the Nobel Memorial Prize in Economic Science. The first one was awarded in 1969, so it's fairly recent. This means that many of the work by the laureates are easily available online. In this week's exercises we will do two things:

1. Scrape Wikipedia to retrieve informations about the location of the Alma Mater(s) of each Nobel Memorial Prize in Economic Sciences laureate. We want to know specifically how many Nobel Memorial Prize winner have never worked in an university in the United States.
2. Retrieve quotes from those people in an automated fashion so that we can pick one at random in a "quote of the day" spirit.

Wikipedia is a great source for this endeavour as the website curators are notably favorable towards scrapers and make little effort to try to deter them. It is therefore a great use case for a first foray into the topic.

### Retrieving the name and wikipedia page of all the laureate

There is a list of all the laureates available [here](https://en.wikipedia.org/wiki/List_of_Nobel_Memorial_Prize_laureates_in_Economics). The URL is "https://en.wikipedia.org/wiki/List_of_Nobel_Memorial_Prize_laureates_in_Economics". Let's just use requests and Beautiful Soup to retrieve all the names and wikipedia page of the laureates.

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_Nobel_Memorial_Prize_laureates_in_Economics"
response = requests.get(url)
soup  = BeautifulSoup(response.content)

rows = soup.find("table", attrs = {"class" : "wikitable"}).find("tbody").find_all("tr")

year = 0
name_cells = []

for row in rows:
    if len(row.find_all("td")) > 0:
        try:
            year = int(row.find_all("td")[0].text)
            name_cells.append([year, row.find_all("td")[2]])
        except:
            name_cells.append([year, row.find_all("td")[1]])

wikipedia_root = "https://en.wikipedia.org"
laureates_infos = {element[1].text.strip("\n") : {"year" : element[0], "webpage" : wikipedia_root + element[1].find("a").get("href")}
                   for element in name_cells}

Now that we have a list with the names and wikipedia pages of all laureates, we need to get inside each of those in order to get the name of their alma mater. The issue is that several of the laureates have several alma mater. Spend a little time looking at several of the pages of winners to see how this is reflected in the HTML.

In this case, extracting the information might be wrapped inside a function that will be applied on the value of the dictionary created at the step before. Please use regular expression to identify the words "Alma Mater" (with and without capital letters for each word)

In [18]:
import re

def extract_alma_mater(url):
    regex = r'([aA]lma\s[mM]ater|[Ee]ducation)'
    wikipedia_root = "https://en.wikipedia.org"
    soup = BeautifulSoup(requests.get(url).content)
    infos = soup.find("table", attrs = {"class": "infobox"})
    for cell in infos.find_all(scope = "row"):
        if re.match(regex, str(cell.text)):
            almas = cell.next_sibling.find_all("a")
            return {alma.get("title"): wikipedia_root + alma.get("href") for alma in almas}

And now, repeat the same to identify in which country this university is located. No need to use regular expressions here.

In [14]:
def extract_country_from_university(url):
    soup = BeautifulSoup(requests.get(url).content)
    infos = soup.find("table", attrs = {"class": "infobox"})
    try:
        for cell in infos.find_all(scope = "row"):
            if str(cell.text) == "Location":
                loc = cell.next_sibling
                regex = r'^([A-Za-z\s]+)'
                return re.findall( regex, loc.text.split(",")[-1])[0]
    except:
        return ''

We are now ready to put everything together. Put everything together to modify the structure in which you store the name of the laureates to contain also the countries of their alma mater.

In [19]:
for laureate in laureates_infos:
    alma_maters = extract_alma_mater(laureates_infos[laureate]["webpage"])
    try:
        laureates_infos[laureate]["Countries"] = [extract_country_from_university(alma_maters[alma]) for alma in alma_maters]
    except:
        laureates_infos[laureate]["Countries"] = []
    if '' in laureates_infos[laureate]["Countries"]: laureates_infos[laureate]["Countries"].remove('')

In [20]:
laureates_infos["Esther Duflo"]

{'year': 2019,
 'webpage': 'https://en.wikipedia.org/wiki/Esther_Duflo',
 'Countries': [' France', ' France', '', ' United States', '']}

We are now ready to perform our analysis. Note however that what we have done could have been done in an more structured fashion had we used *pandas*, the library that enables working with dataframes.

Moreover, you will see that some of the results are not correct and/or available (for example, Esther Duflo has no "Alma Mater" listed on her page (because in her case, it is called "Education") and some universities, such as the University of Leningrad, have no location). This is fine for now but a more comprehensive solution should determine something to work with that.

## Voices from the past

We are now going to collect all the quotes avaiable on Wikiquotes from the Nobel Memorial Prize for Economic Science laureates. In order to do so, we will pass through the list of laureates pages. If there is a link to a quotes page, we will scrape all the quotes there. If not, we will simply ignore it.

When doing so, it usually makes sense to start by writing the code (in a function) that will scrape the quotes and then write the loop that pass on each laureate and identify if it is relevant to scrape it. Write the function that collects all the quotes from a wikiquote page. 

You will see that, for some authors, there are also quotes about the person (rather than quotes from the person). In this simple example, it is OK to collect them as well although in a real setting, we probably will want to avoid keeping those.

If you need a page to serve as example, here is [Ragnar Frisch's wikiquotes page](https://en.wikiquote.org/wiki/Ragnar_Frisch).

In [37]:
def retrieve_quotes(url):
    soup = BeautifulSoup(requests.get(url).content)
    result = []
    quotes = soup.find("span", id="Quotes").parent
    type_next_tag = ""
    while type_next_tag != "h2":
        quotes = quotes.next_sibling
        type_next_tag = quotes.name
        if type_next_tag == "ul":
            indiv_quotes = quotes.find_all("li", recursive = False)
            result += [q.text for q in indiv_quotes]
        if type_next_tag == "dl":
            result[-1] = result[-1] + "\n" + quotes.text
    return result

quotes = retrieve_quotes("https://en.wikiquote.org/wiki/Ragnar_Frisch")
print(quotes[5])

I approached the problem of utility measurement in 1923 during a stay in Paris. There were three objects I had in view :
(I) To point out the choice axioms that are implied when we think of utility as a quantity, and to define utility in a rigorous way by starting from a set of such axioms;
(II) To develop a method of measuring utility statistically;
(III) To apply the method to actual data.
The results of my study along these lines are contained in a paper “Sur un Problème d’Économic Pure”, published in the Series Norsk Matematisk Forenings Skrifter, Serie I, Nr 16, 1926. In this paper, the axiomatics are worked out so far as the static utility concept is concerned. The method of measurement developed is the method of isoquants, which is also outlined in Section 4 below. The statistical data to which the method was applied were sales and price statistics collected by the “Union des Coopérateurs Parisien”. From these data I constructed what I believe can be considered the marginal util

We are now going to write the part that goes through the pages of all laureates, checks whether they have a Wikiquote page (the link is in the left sidebar) and then retrieve the quotes. We can, as we did previously, put the quotes in a dictionary of lists. Try avoiding having lists of size 0.

In [44]:
quotes = {}
for laureate in laureates_infos:
    url = laureates_infos[laureate]["webpage"]
    soup = BeautifulSoup(requests.get(url).content)
    list_element = soup.find(class_ = "wb-otherproject-wikiquote")
    if list_element != None:
        link = list_element.find("a").get("href")
        new_quotes = retrieve_quotes(link)
        if len(new_quotes) > 0:
            quotes[laureate] = new_quotes
        else:
            print(laureate)

Now that all the quotes are collected, we just use a standard random number generator to select one at random. You can either select a quote from a big "bag" containing all quotes but I would rather first choose a laureate and then a quote in order to avoid favouring the laureates who were more prolix (or who have a larger fan-base).

In [50]:
import random
author = random.choice(list(quotes.keys()))
quote = random.choice(quotes[author])

print(quote + "\n" + author)

We economists love to quote Keynes’s final lines in his 1936 General Theory—for the reason that they cater so well to our vanity and self-importance. But to admit the truth, madmen in authority can self generate their own frenzies without needing help from either defunct or avant-garde economists. What establishment economists brew up is as often what the Prince and the Public are already wanting to imbibe. We guys don’t stay in the best club by proffering the views of some past academic crank or academic sage.
Coeditor's Forword in Inside the economist’s mind: conversations with eminent economists (2007)
Paul Samuelson
